In [ ]:
#Data Ingestion
import boto3
from io import StringIO
import pandas as pd
from awsglue.utils import getResolvedOptions
import base64
import sys
from datetime import datetime
import re
import logging
from botocore.config import Config
import psycopg2
from ansible_secret import secret
from awsaccount import AwsAccount
from stssession import StsSession
import concurrent.futures



def create_boto3_client(service, proxy, region):
    try: 

        # support_client = boto3.client('support',region_name='us-west-2')
        support_client = boto3.client(service, config=Config(
            region_name = region,
            proxies={
                'http': proxy,
                'https': proxy
            })) 

        return support_client

    except ClientError as e: 
        logging.error(e)


    if 'afterTime' in params and params['afterTime']!=' ':
        try:
            datetime.strptime(params['afterTime'], '%Y-%m-%d')
            start_time = params['afterTime']
        except ValueError:            
            raise ValueError("Incorrect data format, should be YYYY-MM-DD")
    
    requestStatusCode = response['ResponseMetadata']['HTTPStatusCode']
    
   
def write_to_s3(df,bucket_name,file_name):
    bucket = bucket_name # already created on S3
    location = 's3://'+bucket+'/AWS-SupportCaseData/'
    df.to_csv(location + file_name, index=False)

    
reqd_params =  ['caseIdList',
                'bucket_name',
                'includeResolvedCases',
                'includeCommunications',
                'fieldsrequired',
                'PGUSER',
                'PGDATABASE',
                'PGPORT',
                'PGHOST',
                'PGEncpPASSWORD',
                'AwsEnv',
                "region",
                'HttpProxy',
               'afterTime']
params = getResolvedOptions(sys.argv,reqd_params)
# bucket_name = params['bucket_name']


bucket_name = 'vz-app-itcs-h9vv-non-prod-lambda-deploy-s3bucket'


try:

    secrets_client = create_boto3_client('secretsmanager', params['HttpProxy'], params['region'])
    s = secret(key=bytes(secrets_client.get_secret_value(SecretId=f"GTS.H9VV.{params['region']}.nonprod.Ansible")['SecretString'],'utf-8'))
    dbpassword = s.decrypt(base64.b64decode(params['PGEncpPASSWORD']))

    conn = psycopg2.connect(f"host={params['PGHOST']} port={params['PGPORT']} dbname={params['PGDATABASE']} user={params['PGUSER']} password={dbpassword}")

    def getAwsAccounts(conn):
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as curs:
            print(f'Querying the tblawsaccounts for all accounts')
            curs.execute("select * from tblawsaccounts where env='NONPROD' and is_airgap_acct=false and accountname NOT IN ('itin-np', 'mgc-np','vz-it-np-exhv-itnuc-mgmt', 'mgca-np', 'vz-it-np-h9vv-test01','vz-mp03-np-h9vv-test01')")
            return map(AwsAccount, curs.fetchall())

    
    cases = []
    case_list = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        acct = list(getAwsAccounts(conn))
               
        for awsAccount in acct:
            try: 
                stsSession = StsSession(conn, awsAccount, f'ucs_glue', params['HttpProxy'])
                support_client =  stsSession.getRemoteClient('support',params['region'], params['HttpProxy'])

                fieldsrequired = params['fieldsrequired'].split(',')
                resolved_cases = bool(params['includeResolvedCases'])
                datetime.strptime(params['afterTime'], '%Y-%m-%d')
                start_time = params['afterTime']
                commns = bool(params['includeCommunications'])
                response = support_client.describe_cases(
                includeResolvedCases=resolved_cases,
                includeCommunications=commns,
                afterTime=start_time,
                maxResults=100)
                df = pd.DataFrame(response['cases'])
                list_body = []
                if len(df)>0:
                    for index in range(len(df)):
                        bodies = ''
                        for items in (df.loc[index,'recentCommunications'])['communications']:
                            bodies+=items['body'] + ' '
                        list_body.append(bodies)
                    df['Comments'] = list_body
                    df = df.drop(['recentCommunications'],axis=1)
                    case_list.append(df)
                    
                

            except Exception as e:
                logging.info(e)
            
        df_case_list = pd.concat(case_list, ignore_index=True)    
                
        if isinstance(df_case_list,pd.DataFrame):
            file_name = 'AWSSupportcases-'+ datetime.now().strftime("D%Y-%m-%dT%H:%M:%S")+'.csv'
            write_to_s3(df_case_list,bucket_name,file_name)
   

except Exception as e:
    logging.error(e)

In [3]:
import os
os.getcwd()

'C:\\Users\\Naveen Vatsal'